In [ ]:
import os
import sys
from dotenv import load_dotenv
import time
import pandas as pd

load_dotenv()

# Set the download path for demonstration and ensure the folder exists.
download_path = os.getenv("ASA_DOWNLOAD_PATH")
os.makedirs(download_path, exist_ok=True)

git_path = os.getenv("GIT_FOLDER")
cv3_path = os.getenv("CV3_FOLDER")
sys.path.append(git_path)
sys.path.append(cv3_path)

In [ ]:
from cerulean_cloud.cloud_function_asa.utils.analyzer import (
    AISAnalyzer,
)

In [ ]:
# Import functions from your script.
from asa_analysis.evaluation.source_analyzer_evaluation import (
    process_groundtruth_on_analyzer,
)

In [ ]:
# Load hitl CSV for vessel and infrastructure groundtruth.
csv_path = r"../slick_to_source_2025-3-20.csv"
hitl_df = pd.read_csv(csv_path)

# Extract groundtruth lists for infrastructure and vessels.
vessel_groundtruth = hitl_df[(hitl_df["type"] == 1) & (hitl_df["hitl_verification"])]

if "slick" in vessel_groundtruth.columns:
    vessel_groundtruth = vessel_groundtruth.rename(columns={"slick": "slick_id"})

DOWNLOAD VESSEL FINETUNING ON `HOURS_BEFORE`, `HOURS_AFTER`, and `AIS_BUFFER`

In [ ]:
# before_after = [[4,2], [6,4], [8,4], [10,6], [12,6], [14,8], [16,8]]
# before_after2 = [[10,4], [8,6], [9,5]]
before_after3 = [[1, 1]]
results_over_intervals = []

In [ ]:
buffer = 20000
for before, after in before_after3:
    kwargs = {"hours_before": before, "hours_after": after, "ais_buffer": buffer}
    start_time = time.time()
    results_vessel_local = process_groundtruth_on_analyzer(
        AISAnalyzer,
        vessel_groundtruth,
        analyzer_params=kwargs,
        reuse_ais_trajectories=False,
        reuse_ais_gdf=True,
    )
    end_time = time.time()
    results_vessel_local["runtime"] = end_time - start_time
    results_vessel_local.drop(columns=["geometry"]).to_csv(
        f"saves/vessel_results_{before}_{after}_{buffer}.csv", index=False
    )

In [ ]:
buffers = [0, 100, 1000, 10000, 15000, 20000, 30000, 50000]
for buffer in buffers:
    kwargs = {"hours_before": 10, "hours_after": 6, "ais_buffer": buffer}
    start_time = time.time()
    results_vessel_local = process_groundtruth_on_analyzer(
        AISAnalyzer,
        vessel_groundtruth,
        analyzer_params=kwargs,
        reuse_ais_trajectories=False,
        reuse_ais_gdf=True,
    )
    end_time = time.time()
    results_vessel_local["runtime"] = end_time - start_time
    results_vessel_local.drop(columns=["geometry"]).to_csv(
        f"saves/vessel_results_{before}_{after}_{buffer}.csv", index=False
    )